# Part 8.6: GGUF vs ExecuTorch — Edge Deployment for SLMs

## Choosing the Right Deployment Format

In Part 8.5, we learned the fundamentals of ExecuTorch. Now we'll:

1. **Compare GGUF and ExecuTorch** — Both are for local inference, but serve different purposes
2. **Apply ExecuTorch to an SLM** — Export a real language model for edge deployment
3. **Understand the trade-offs** — When to use each format

**Learning Objectives:**
- Understand the key differences between GGUF and ExecuTorch
- Know when to choose each deployment format
- Export a Small Language Model to ExecuTorch format
- Handle LLM-specific challenges in edge deployment

---

## 1. GGUF vs ExecuTorch: Understanding the Difference

Both GGUF and ExecuTorch enable local, on-device inference without cloud dependencies. But they target **fundamentally different environments**.

### 1.1 GGUF (llama.cpp)

**Target:** Desktops, laptops, servers — machines with a full operating system

**How it works:**
- Quantized model weights stored in a single file
- llama.cpp runtime (C/C++) loads and executes
- Optimized for CPU inference on x86/ARM with SIMD instructions
- Can use GPU acceleration via Metal, CUDA, Vulkan

**Runtime requirements:**
- Needs llama.cpp binary (~10-50MB compiled)
- Runs as a standalone process on Linux/macOS/Windows
- Full OS environment assumed

**Typical use cases:**
- Running LLMs on your MacBook
- Local chatbots on desktop
- Privacy-focused deployments
- Hobbyist/developer experimentation

### 1.2 ExecuTorch

**Target:** Edge devices — phones, watches, IoT, embedded systems

**How it works:**
- Model compiled to `.pte` format with AOT optimization
- Minimal C++ runtime (~1MB or less)
- Designed to embed inside apps (iOS/Android) or run on bare-metal
- Hardware delegation to NPUs, DSPs, neural accelerators

**Runtime requirements:**
- No OS required in some cases (bare-metal possible)
- Integrates into mobile app binaries
- Designed for extreme resource constraints

**Typical use cases:**
- On-device AI in mobile apps (keyboard prediction, photo enhancement)
- Wearables (smartwatch health monitoring)
- IoT sensors (anomaly detection)
- Automotive (driver assistance)
- Anywhere internet connectivity isn't guaranteed

### 1.3 Side-by-Side Comparison

| Aspect | GGUF / llama.cpp | ExecuTorch |
|--------|------------------|------------|
| **Primary target** | Desktop/laptop/server | Mobile/embedded/IoT |
| **Minimum RAM** | ~2GB+ realistic | Can work with 128MB |
| **Binary size** | ~10-50MB runtime | ~1MB runtime possible |
| **OS required** | Yes (Linux/macOS/Windows) | No (can run bare-metal) |
| **Hardware acceleration** | CPU SIMD, optional GPU | NPU, DSP, Neural Engine |
| **Integration model** | Standalone process | Embedded in app |
| **Python needed** | No (at runtime) | No (at runtime) |
| **Quantization** | GGUF formats (Q4_K_M, etc.) | PTQ, QAT, dynamic/static |
| **Primary use case** | LLMs on personal computers | AI features in apps/devices |

### 1.4 The Key Insight

**GGUF assumes you have a computer.** It's optimized for the kind of machine where you'd run Ollama or a local server.

**ExecuTorch assumes you might not.** It's optimized for the kind of device where every megabyte matters and you're shipping inside an app binary.

### Real-World Example

Imagine you fine-tuned a small assistant model:

| Deployment Path | User Experience |
|-----------------|----------------|
| **GGUF** | User downloads Ollama, pulls your model, runs it in terminal or via API. Works great on a MacBook. |
| **ExecuTorch** | You embed the model inside an iOS/Android app. User installs from App Store. Model runs entirely on-device, no setup required, works offline, uses the phone's neural accelerator. |

Same model, completely different deployment story.

### 1.5 Decision Framework

**Choose GGUF when:**
- Users have desktops/laptops
- RAM is 4GB+ available
- Running as a standalone application or server
- You want quick experimentation with different quantization levels
- CPU inference is acceptable

**Choose ExecuTorch when:**
- Deploying to mobile apps (iOS/Android)
- Target device has <2GB RAM
- Need to leverage NPU/Neural Engine
- Model must be embedded in app binary
- Offline functionality is critical
- Battery efficiency matters

---

## 2. ExecuTorch for LLMs: Special Considerations

Deploying language models to edge devices presents unique challenges compared to simple classifiers.

### 2.1 LLM-Specific Challenges

| Challenge | Description | Solution |
|-----------|-------------|----------|
| **Model Size** | Even small LLMs are 100MB-1GB+ | Aggressive quantization (4-bit, 8-bit) |
| **KV Cache** | Grows with sequence length | Fixed cache size, sliding window |
| **Autoregressive Generation** | Token-by-token loop | Optimized generation kernels |
| **Tokenization** | Needs tokenizer on device | Export tokenizer alongside model |
| **Dynamic Shapes** | Variable sequence lengths | Use dynamic shape export or pad |

### 2.2 ExecuTorch LLM Solutions

Meta has developed specific optimizations for LLMs on ExecuTorch:

**1. Custom Attention Kernels**
- Optimized SDPA (Scaled Dot-Product Attention) for mobile
- Efficient KV cache management

**2. Quantization Support**
- 8-bit dynamic quantization
- 4-bit weight quantization
- Mixed precision strategies

**3. Hardware Delegation**
- CoreML for Apple Neural Engine
- Qualcomm QNN for Snapdragon NPUs
- XNNPACK for optimized CPU execution

**4. Memory Optimization**
- Tensor memory planning
- Efficient buffer reuse

---

## 3. Hands-On: Exporting an SLM to ExecuTorch

Let's export a real Small Language Model. We'll use a small model to demonstrate the workflow.

### 3.1 Environment Setup

In [1]:
# Install required packages
!pip install executorch transformers torch accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.0/543.0 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 5.0 MB/s eta 0:00:00


In [2]:
# Verify imports
import torch
print(f"PyTorch version: {torch.__version__}")

try:
    from torch.export import export
    from executorch.exir import to_edge
    print("✓ ExecuTorch imported successfully!")
except ImportError as e:
    print(f"✗ ExecuTorch import error: {e}")

try:
    from transformers import AutoModelForCausalLM, AutoTokenizer
    print("✓ Transformers imported successfully!")
except ImportError as e:
    print(f"✗ Transformers import error: {e}")

PyTorch version: 2.9.0+cu126


✓ ExecuTorch imported successfully!
✓ Transformers imported successfully!


### 3.2 Approach 1: Export a Minimal GPT-2 Style Model

For demonstration purposes, we'll first create a minimal transformer-style model that's easy to export. This helps understand the process before tackling full LLMs.

In [3]:
import torch
import torch.nn as nn
import math

class MiniTransformerBlock(nn.Module):
    """A minimal transformer block for demonstration."""

    def __init__(self, embed_dim=64, num_heads=4):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.GELU(),
            nn.Linear(embed_dim * 4, embed_dim)
        )

    def forward(self, x):
        # Self-attention with residual
        attn_out, _ = self.attention(x, x, x, need_weights=False)
        x = self.norm1(x + attn_out)

        # FFN with residual
        ffn_out = self.ffn(x)
        x = self.norm2(x + ffn_out)

        return x


class MiniLM(nn.Module):
    """
    A minimal language model for ExecuTorch demonstration.

    This is NOT a real LLM - it's a simplified architecture
    to demonstrate the export process.
    """

    def __init__(self, vocab_size=1000, embed_dim=64, num_heads=4, num_layers=2, max_seq_len=32):
        super().__init__()
        self.embed_dim = embed_dim
        self.max_seq_len = max_seq_len

        # Token embedding
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)

        # Positional embedding
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)

        # Transformer blocks
        self.blocks = nn.ModuleList([
            MiniTransformerBlock(embed_dim, num_heads)
            for _ in range(num_layers)
        ])

        # Output projection
        self.ln_f = nn.LayerNorm(embed_dim)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)

    def forward(self, input_ids):
        batch_size, seq_len = input_ids.shape

        # Get embeddings
        positions = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)

        # Apply transformer blocks
        for block in self.blocks:
            x = block(x)

        # Output projection
        x = self.ln_f(x)
        logits = self.lm_head(x)

        return logits


# Create the model
model = MiniLM(
    vocab_size=1000,
    embed_dim=64,
    num_heads=4,
    num_layers=2,
    max_seq_len=32
)
model.eval()

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Model created: MiniLM")
print(f"Parameters: {num_params:,} ({num_params/1e6:.2f}M)")

# Test forward pass
sample_input = torch.randint(0, 1000, (1, 16))  # batch=1, seq_len=16
with torch.no_grad():
    output = model(sample_input)
print(f"Input shape:  {sample_input.shape}")
print(f"Output shape: {output.shape}  (batch, seq_len, vocab_size)")

Model created: MiniLM
Parameters: 230,144 (0.23M)
Input shape:  torch.Size([1, 16])
Output shape: torch.Size([1, 16, 1000])  (batch, seq_len, vocab_size)


### 3.3 Export MiniLM to ExecuTorch

In [4]:
from torch.export import export
from executorch.exir import to_edge

# Prepare example input
# Note: We use a fixed sequence length for export
example_input = (torch.randint(0, 1000, (1, 16)),)

print("Step 1: Exporting model...")
try:
    exported_program = export(model, example_input)
    print("✓ Export successful!")
except Exception as e:
    print(f"✗ Export failed: {e}")
    raise

Step 1: Exporting model...
✓ Export successful!


In [5]:
# Convert to Edge dialect
print("Step 2: Converting to Edge dialect...")
try:
    edge_program = to_edge(exported_program)
    print("✓ Edge conversion successful!")
except Exception as e:
    print(f"✗ Edge conversion failed: {e}")
    raise

Step 2: Converting to Edge dialect...
✓ Edge conversion successful!


In [6]:
# Compile to ExecuTorch
print("Step 3: Compiling to ExecuTorch...")
try:
    executorch_program = edge_program.to_executorch()
    print("✓ ExecuTorch compilation successful!")
    print(f"  Buffer size: {len(executorch_program.buffer):,} bytes ({len(executorch_program.buffer)/1024:.2f} KB)")
except Exception as e:
    print(f"✗ Compilation failed: {e}")
    raise

Step 3: Compiling to ExecuTorch...
✓ ExecuTorch compilation successful!
  Buffer size: 945,664 bytes (923.50 KB)


In [7]:
import os

# Save the .pte file
pte_path = "mini_lm.pte"

with open(pte_path, "wb") as f:
    f.write(executorch_program.buffer)

file_size = os.path.getsize(pte_path)
print(f"Step 4: Saved to {pte_path}")
print(f"  File size: {file_size:,} bytes ({file_size/1024:.2f} KB)")

Step 4: Saved to mini_lm.pte
  File size: 945,664 bytes (923.50 KB)


### 3.4 Run Inference with ExecuTorch

In [8]:
from executorch.runtime import Runtime

# Load the compiled model
runtime = Runtime.get()
program = runtime.load_program(pte_path)
method = program.load_method("forward")

print("✓ Model loaded!")

# Run inference
test_input = torch.randint(0, 1000, (1, 16))
outputs = method.execute([test_input])

print(f"\nInference Results:")
print(f"Input shape:  {test_input.shape}")
print(f"Output shape: {outputs[0].shape}")

[program.cpp:153] InternalConsistency verification requested but not available


✓ Model loaded!

Inference Results:
Input shape:  torch.Size([1, 16])
Output shape: torch.Size([1, 16, 1000])


### 3.5 Verify Results Match PyTorch

In [9]:
# Compare PyTorch vs ExecuTorch outputs
test_input = torch.randint(0, 1000, (1, 16))

# PyTorch
with torch.no_grad():
    pytorch_output = model(test_input)

# ExecuTorch
executorch_output = method.execute([test_input])[0]

# Compare
are_close = torch.allclose(pytorch_output, executorch_output, atol=1e-4)
max_diff = torch.max(torch.abs(pytorch_output - executorch_output)).item()

print("Comparison: PyTorch vs ExecuTorch")
print("=" * 50)
print(f"Outputs match (atol=1e-4): {are_close}")
print(f"Maximum difference: {max_diff:.6f}")

if are_close:
    print("\n✓ ExecuTorch produces equivalent results!")

Comparison: PyTorch vs ExecuTorch
Outputs match (atol=1e-4): True
Maximum difference: 0.000001

✓ ExecuTorch produces equivalent results!


---

## 4. Approach 2: Exporting a HuggingFace Model

Now let's try exporting a real (small) model from HuggingFace. This is more complex due to the model architecture.

### 4.1 The Challenge with HuggingFace Models

HuggingFace models have complexities that make direct export challenging:

1. **Dynamic control flow** — KV caching, variable attention masks
2. **Custom operations** — Rotary embeddings, specialized attention
3. **Large size** — Even small models are 100MB+
4. **Generation loop** — Autoregressive decoding is outside the model

**Solutions:**
- Use `torch.export` with strict=False for more flexibility
- Wrap the model to handle specific inputs
- Use ExecuTorch's LLM-specific utilities (when available)

### 4.2 Export Strategy: Wrapper Approach

We'll wrap a HuggingFace model to make it export-friendly.


AutoTokenizer loads GPT-2 tokenizer.

GPT-2 often has no pad token, so you set pad_token = eos_token to avoid padding errors.

AutoModelForCausalLM loads a causal language model (next-token prediction).

use_cache=False disables KV-cache (past key/values). That’s important because:

KV-cache introduces extra outputs /
state that complicate export.

eval() puts the model into inference mode (no dropout etc.).




In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

# Use a very small model for demonstration
# Options: "sshleifer/tiny-gpt2", "hf-internal-testing/tiny-random-gpt2"
model_name = "sshleifer/tiny-gpt2"

print(f"Loading model: {model_name}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model
hf_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    use_cache=False  # Disable KV cache for simpler export
)
hf_model.eval()

# Count parameters
num_params = sum(p.numel() for p in hf_model.parameters())
print(f"✓ Model loaded!")
print(f"  Parameters: {num_params:,}")

Loading model: sshleifer/tiny-gpt2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

✓ Model loaded!
  Parameters: 102,714


In [11]:
# Test the model
test_text = "Hello world"
inputs = tokenizer(test_text, return_tensors="pt")

with torch.no_grad():
    outputs = hf_model(**inputs)

print(f"Input: '{test_text}'")
print(f"Input IDs shape: {inputs['input_ids'].shape}")
print(f"Output logits shape: {outputs.logits.shape}")

Input: 'Hello world'
Input IDs shape: torch.Size([1, 2])
Output logits shape: torch.Size([1, 2, 50257])


### 4.3 Create Export-Friendly Wrapper


Hugging Face models usually return a complex object (ModelOutput) with many fields (logits, hidden states, cache, etc.). Export tools prefer:

- simple tensor inputs
- simple tensor outputs

So this wrapper forces:

- input: input_ids only
- output: logits tensor only

In [12]:
class ExportableGPT2(nn.Module):
    """
    Wrapper to make GPT-2 style models export-friendly.

    Key changes:
    - Takes only input_ids (no attention_mask for simplicity)
    - Returns only logits tensor
    - Disables KV cache
    """

    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids):
        # Create attention mask (all ones for simplicity)
        attention_mask = torch.ones_like(input_ids)

        # Forward pass
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            use_cache=False
        )

        # Return only logits
        return outputs.logits


# Create wrapper
exportable_model = ExportableGPT2(hf_model)
exportable_model.eval()

ExportableGPT2(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 2)
      (wpe): Embedding(1024, 2)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-1): 2 x GPT2Block(
          (ln_1): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D(nf=6, nx=2)
            (c_proj): Conv1D(nf=2, nx=2)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D(nf=8, nx=2)
            (c_proj): Conv1D(nf=2, nx=8)
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((2,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=2, out_features=50257, bias=False)
  )
)

You create attention_mask = ones so the model treats all tokens as real tokens (no padding). That’s fine for demo/export, but in real usage you usually want a real attention mask.

Then you test it:

In [13]:
# Test wrapper
test_ids = torch.randint(0, tokenizer.vocab_size, (1, 8))
with torch.no_grad():
    output = exportable_model(test_ids)
print(f"Wrapper test - Input: {test_ids.shape}")
print(f"Output: {output.shape}")

Wrapper test - Input: torch.Size([1, 8])
Output: torch.Size([1, 8, 50257])


If it prints shapes correctly, the wrapper works.

### 4.4 Export the Wrapped Model

In [14]:
from torch.export import export
from executorch.exir import to_edge

# Prepare example input with fixed sequence length
seq_length = 16
example_input = (torch.randint(0, tokenizer.vocab_size, (1, seq_length)),)

print(f"Exporting with sequence length: {seq_length}")
print("This may take a moment...\n")

# Step 1: Export
print("Step 1: Exporting...")
exported = export(exportable_model, example_input)
print("✓ Export successful!")

# Step 2: Convert to Edge
print("Step 2: Converting to Edge...")
edge = to_edge(exported)
print("✓ Edge conversion successful!")

# Step 3: Compile to ExecuTorch
print("Step 3: Compiling to ExecuTorch...")
et_program = edge.to_executorch()
print("✓ Compilation successful!")

Exporting with sequence length: 16
This may take a moment...

Step 1: Exporting...


model.safetensors:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

✓ Export successful!
Step 2: Converting to Edge...
✓ Edge conversion successful!
Step 3: Compiling to ExecuTorch...
✓ Compilation successful!


torch.export.export(...)
- Captures a static computation graph from your model given example_input.
- This is why you pick a fixed seq_length here.

to_edge(exported)
- Converts the graph into an “edge-safe” representation (EXIR edge dialect).
- Prepares it for mobile/runtime constraints.

to_executorch()
- Compiles it into an ExecuTorch runnable artifact.

In [16]:
# Step 4: Save
pte_path = "tiny_gpt2.pte"
with open(pte_path, "wb") as f:
  f.write(et_program.buffer)

file_size = os.path.getsize(pte_path)
print(f"\n✓ Saved to {pte_path}")
print(f"  File size: {file_size:,} bytes ({file_size/1024/1024:.2f} MB)")



✓ Saved to tiny_gpt2.pte
  File size: 1,490,996 bytes (1.42 MB)


✅ Now tiny_gpt2.pte is the file you’d ship into a mobile/edge app.

### 4.5 Alternative: Using ExecuTorch's LLM Utilities

For production LLM deployment, ExecuTorch provides specialized utilities. Here's the conceptual approach:

In [17]:
# This is conceptual code showing the production approach
# Actual implementation requires additional ExecuTorch LLM packages

CONCEPTUAL_CODE = """
# Production ExecuTorch LLM Export (Conceptual)
# =============================================

from executorch.examples.models.llama2 import Llama2Model
from executorch.exir import to_edge
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner

# 1. Load model with ExecuTorch's optimized wrapper
model = Llama2Model.from_pretrained("path/to/model")

# 2. Export with LLM-specific settings
exported = torch.export.export(model, example_inputs)

# 3. Convert to edge with hardware delegation
edge = to_edge(exported)

# 4. Apply XNNPACK optimization for CPU
edge = edge.to_backend(XnnpackPartitioner())

# 5. Compile with quantization
from executorch.exir import EdgeCompileConfig
config = EdgeCompileConfig(_check_ir_validity=False)
et_program = edge.to_executorch(config)

# 6. Save
with open("llama2.pte", "wb") as f:
    f.write(et_program.buffer)
"""

print("Production ExecuTorch LLM Export Approach:")
print("=" * 50)
print(CONCEPTUAL_CODE)

Production ExecuTorch LLM Export Approach:

# Production ExecuTorch LLM Export (Conceptual)
# =============================================

from executorch.examples.models.llama2 import Llama2Model
from executorch.exir import to_edge
from executorch.backends.xnnpack.partition.xnnpack_partitioner import XnnpackPartitioner

# 1. Load model with ExecuTorch's optimized wrapper
model = Llama2Model.from_pretrained("path/to/model")

# 2. Export with LLM-specific settings
exported = torch.export.export(model, example_inputs)

# 3. Convert to edge with hardware delegation
edge = to_edge(exported)

# 4. Apply XNNPACK optimization for CPU
edge = edge.to_backend(XnnpackPartitioner())

# 5. Compile with quantization
from executorch.exir import EdgeCompileConfig
config = EdgeCompileConfig(_check_ir_validity=False)
et_program = edge.to_executorch(config)

# 6. Save
with open("llama2.pte", "wb") as f:
    f.write(et_program.buffer)



---

## 5. Quantization for Edge Deployment

Quantization is critical for edge deployment. Let's explore the options.

### 5.1 Quantization Options Comparison

| Method | Precision | Size Reduction | Accuracy Impact | Use Case |
|--------|-----------|----------------|-----------------|----------|
| **FP32** | 32-bit | 1x (baseline) | None | Development |
| **FP16** | 16-bit | 2x | Minimal | GPU inference |
| **INT8 Dynamic** | 8-bit | 4x | Low | CPU inference |
| **INT8 Static** | 8-bit | 4x | Low | Calibrated inference |
| **INT4** | 4-bit | 8x | Moderate | Extreme compression |

### 5.2 Applying Dynamic Quantization

In [18]:
# Dynamic quantization example with our MiniLM
from torch.ao.quantization import quantize_dynamic

# Create a fresh model
model_fp32 = MiniLM(
    vocab_size=1000,
    embed_dim=64,
    num_heads=4,
    num_layers=2,
    max_seq_len=32
)
model_fp32.eval()

# Apply dynamic quantization
model_int8 = quantize_dynamic(
    model_fp32,
    {nn.Linear},  # Quantize Linear layers
    dtype=torch.qint8
)

print("Dynamic Quantization Applied!")
print("\nOriginal model (FP32):")
print(f"  Size estimate: {sum(p.numel() * 4 for p in model_fp32.parameters()):,} bytes")

print("\nQuantized model (INT8):")
print(f"  Size estimate: ~{sum(p.numel() for p in model_fp32.parameters()):,} bytes")

Dynamic Quantization Applied!

Original model (FP32):
  Size estimate: 920,576 bytes

Quantized model (INT8):
  Size estimate: ~230,144 bytes


/tmp/ipython-input-2049492293.py:15: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_int8 = quantize_dynamic(


In [19]:
# Compare outputs
test_input = torch.randint(0, 1000, (1, 16))

with torch.no_grad():
    output_fp32 = model_fp32(test_input)
    output_int8 = model_int8(test_input)

# Check difference
max_diff = torch.max(torch.abs(output_fp32 - output_int8)).item()
mean_diff = torch.mean(torch.abs(output_fp32 - output_int8)).item()

print("Quantization Impact:")
print(f"  Max difference:  {max_diff:.6f}")
print(f"  Mean difference: {mean_diff:.6f}")
print("\nSmall differences are expected and usually acceptable!")

Quantization Impact:
  Max difference:  0.033581
  Mean difference: 0.007226

Small differences are expected and usually acceptable!


---

## 6. Complete Workflow Summary

Here's the complete workflow for deploying an SLM to edge devices:

In [20]:
# ============================================================
# COMPLETE SLM TO EDGE DEPLOYMENT WORKFLOW
# ============================================================

import torch
import torch.nn as nn
from torch.export import export
from executorch.exir import to_edge
import os

def deploy_model_to_edge(model, example_input, output_path, model_name="model"):
    """
    Complete pipeline to deploy a PyTorch model to edge devices.

    Args:
        model: PyTorch nn.Module (must be in eval mode)
        example_input: Tuple of example input tensors
        output_path: Path to save the .pte file
        model_name: Name for logging

    Returns:
        dict with deployment info
    """
    print(f"Deploying {model_name} to edge...")
    print("=" * 50)

    # Ensure eval mode
    model.eval()

    # Step 1: Export
    print("[1/4] Exporting model...")
    exported_program = export(model, example_input)
    print("      ✓ Export complete")

    # Step 2: Convert to Edge
    print("[2/4] Converting to Edge dialect...")
    edge_program = to_edge(exported_program)
    print("      ✓ Edge conversion complete")

    # Step 3: Compile to ExecuTorch
    print("[3/4] Compiling to ExecuTorch...")
    et_program = edge_program.to_executorch()
    print("      ✓ Compilation complete")

    # Step 4: Save
    print("[4/4] Saving .pte file...")
    with open(output_path, "wb") as f:
        f.write(et_program.buffer)

    file_size = os.path.getsize(output_path)
    print(f"      ✓ Saved to {output_path}")

    # Summary
    print("\n" + "=" * 50)
    print("DEPLOYMENT COMPLETE")
    print("=" * 50)

    info = {
        "model_name": model_name,
        "output_path": output_path,
        "file_size_bytes": file_size,
        "file_size_kb": file_size / 1024,
        "file_size_mb": file_size / 1024 / 1024
    }

    print(f"Model: {model_name}")
    print(f"Output: {output_path}")
    print(f"Size: {info['file_size_kb']:.2f} KB ({info['file_size_mb']:.2f} MB)")

    return info


# Example usage
print("\n" + "#" * 60)
print("# EXAMPLE: Deploy MiniLM to Edge")
print("#" * 60 + "\n")

# Create model
model = MiniLM(vocab_size=1000, embed_dim=64, num_heads=4, num_layers=2)
model.eval()

# Deploy
info = deploy_model_to_edge(
    model=model,
    example_input=(torch.randint(0, 1000, (1, 16)),),
    output_path="mini_lm_edge.pte",
    model_name="MiniLM"
)


############################################################
# EXAMPLE: Deploy MiniLM to Edge
############################################################

Deploying MiniLM to edge...
[1/4] Exporting model...
      ✓ Export complete
[2/4] Converting to Edge dialect...
      ✓ Edge conversion complete
[3/4] Compiling to ExecuTorch...
      ✓ Compilation complete
[4/4] Saving .pte file...
      ✓ Saved to mini_lm_edge.pte

DEPLOYMENT COMPLETE
Model: MiniLM
Output: mini_lm_edge.pte
Size: 923.50 KB (0.90 MB)


---

## 7. Key Takeaways

### GGUF vs ExecuTorch

| When to Use | GGUF | ExecuTorch |
|-------------|------|------------|
| Target device | Desktop/laptop | Mobile/embedded |
| User setup | Downloads app/binary | Installs from app store |
| Integration | Standalone process | Embedded in app |
| Hardware | CPU (+ optional GPU) | NPU, Neural Engine |

### LLM Edge Deployment Challenges

1. **Model size** → Quantization (INT8, INT4)
2. **Dynamic shapes** → Fixed sequence lengths or padding
3. **KV cache** → Specialized handling or disabled
4. **Tokenization** → Bundle tokenizer with model

### The Export Pipeline

```
PyTorch Model
     ↓ torch.export()
Exported Program (ATen)
     ↓ to_edge()
Edge Program
     ↓ to_executorch()
.pte File → Edge Device
```

### Production Considerations

- Use ExecuTorch's LLM-specific utilities for production
- Apply hardware delegates (XNNPACK, CoreML, QNN) for acceleration
- Test thoroughly on target devices
- Consider memory constraints at every step

---

## References

- [ExecuTorch Documentation](https://pytorch.org/executorch/stable/index.html)
- [ExecuTorch LLM Examples](https://github.com/pytorch/executorch/tree/main/examples/models)
- [llama.cpp / GGUF](https://github.com/ggerganov/llama.cpp)
- [PyTorch Quantization](https://pytorch.org/docs/stable/quantization.html)
- [HuggingFace Transformers](https://huggingface.co/docs/transformers/)